# Machine learning with GIS based housing data

Experimenting with GIS based housing data.

### Import packages

In [1]:
import json
import math
import warnings
warnings.filterwarnings(action="ignore")

from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from IPython.display import display_html

Definde constants.

- ``PATH``: Path to the base data folder
- ``COUNT_RADIUS``: Maximum distance for count based features
- ``K_FOLDS``: Number of folds to perform for cross validation

In [2]:
PATH = "C:/Users/Tim/.keras/datasets/wikipedia_real_estate/"
COUNT_RADIUS = 3500  # in meters
K_FOLDS = 5

### Defining useful functions

In [3]:
def find_coord(x, df):
    """Returns id, latitude and longitude for property with given id"""
    
    _id, lat, long = x[0], x[1], x[2]
    row = df[df["_id"] == _id].iloc[0]
    return row["_id"], row["latitude"], row["longitude"]

In [4]:
def make_train_test(df):
    """Returns train/test sets along with column names and df for saving errors"""

    X = df.drop(["PROPERTYZIP", "MUNICODE", "SCHOOLCODE", "NEIGHCODE", "SALEDATE", "SALEPRICE",
                 "FAIRMARKETTOTAL", "latitude", "longitude", "SALEYEAR"], axis=1)

    # save col names for later
    X_columns = list(X.columns)
    # remove id from col list, since it will be filtered out later
    X_columns.remove("_id")
    X = X.to_numpy()

    y = df["SALEPRICE"].to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42)

    # save ids for later
    train_ids = [x[0] for x in X_train]
    test_ids = [x[0] for x in X_test]
    X_train = X_train[:, 1:]  # remove first column (id)
    X_test = X_test[:, 1:]    # remove first column (id)

    X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
        X_train, y_train, test_size=0.25, random_state=42)

    print(f"{X_train.shape}: {X_train_train.shape} + {X_train_val.shape}")
    print(f"{y_train.shape}: {y_train_train.shape} + {y_train_val.shape}")
    print(X_test.shape)
    print(y_test.shape)

    # create error df
    error_df = pd.DataFrame(
        data={"id": test_ids, "lat": [0]*len(test_ids), "long": [0]*len(test_ids)})
    error_df = error_df.apply(lambda x: find_coord(
        x, df), axis=1, result_type='broadcast')
    error_df.head(10)

    return X_columns, [X, y, X_train, X_test, y_train, y_test, X_train_train, X_train_val, y_train_train, y_train_val], error_df

In [5]:
def mean_absolute_percentage_error(y_true, y_pred):
    """Returns MAPE"""
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
def get_metrics(y_true, y_pred, print_out=True):
    """Returns MAE, RMSE, MAPE and R^2"""
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r_squared = r2_score(y_true, y_pred)

    if print_out:
        print(f"MAE:  {round(mae)}")
        print(f"RMSE: {round(rmse)}")
        print(f"MAPE: {round(mape, 2)}%")
        print(f"R^2:  {round(r_squared, 3)}")

    return mae, rmse, mape, r_squared

In [7]:
def cross_validation(estimator, X, y):
    """Returns and prints cross validated MAE, RMSE, MAPE and R^2"""
    
    maes, rmses, mapes, r_squareds = [], [], [], []
    X_cv = X[:, 1:]  # remove "_id" column

    kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
    for train_index, test_index in tqdm(kf.split(X_cv), total=5):
        X_train, X_test = X_cv[train_index], X_cv[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        if "linear_model" in str(type(estimator)):
            estimator.fit(X=X_train, y=y_train)
        else:
            estimator.fit(X=X_train, y=y_train, verbose=False)

        y_pred_cv = estimator.predict(X_test)
        mae, rmse, mape, r_squared = get_metrics(y_test, y_pred_cv, print_out=False)
        maes.append(mae)
        rmses.append(rmse)
        mapes.append(mape)
        r_squareds.append(r_squared)
    
    mae_cv, rmse_cv = round(np.mean(maes)), round(np.mean(rmses))
    mape_cv, r_squared_cv = round(np.mean(mapes), 2), round(np.mean(r_squareds), 3)
    
    print(f"MAE:  {mae_cv}")
    print(f"RMSE: {rmse_cv}")
    print(f"MAPE: {mape_cv}%")
    print(f"R^2:  {r_squared_cv}")
    
    return mae_cv, rmse_cv, mape_cv, r_squared_cv

Load structured data with added GIS features.

In [8]:
structured_gis = pd.read_csv(PATH+f"structured_gis_category_features_{COUNT_RADIUS}_radius.csv")
print(structured_gis.shape)
structured_gis.head(10)

(9556, 136)


,_id,PROPERTYZIP,MUNICODE,SCHOOLCODE,NEIGHCODE,LOTAREA,SALEDATE,SALEPRICE,FAIRMARKETTOTAL,STORIES,...,apartment_buildings_dist,apartment_buildings_count,faith-based_facilities_dist,faith-based_facilities_count,restaurants_dist,restaurants_count,community_nonprofit_orgs_dist,community_nonprofit_orgs_count,bus_stops_dist,bus_stops_count
0,427021,15037,908,16,90803,10276,04-21-2017,179900.0,157200,2.0,...,1295.712181,2,430.854222,3,766.161991,11,430.131485,34,4100.223256,0
1,428296,15106,850,7,85002,9375,12-27-2019,185000.0,180400,1.5,...,1855.096394,29,1418.878172,23,556.515602,79,132.952974,140,952.974317,193
2,428307,15237,927,27,92705,14827,01-17-2017,226400.0,167900,1.0,...,720.786679,21,921.939028,16,625.040058,73,322.297063,105,982.293948,122
3,230894,15236,877,4,87702,8206,08-17-2018,140500.0,122800,1.0,...,755.689725,30,924.038244,27,761.305188,99,23.971638,145,151.018551,174
4,231082,15241,950,42,95002,13050,07-17-2020,365000.0,255300,2.0,...,1274.896936,6,1823.857016,20,890.492128,54,31.019148,118,1132.092745,90
5,429294,15220,120,47,12001,10553,04-04-2017,203500.0,152900,1.0,...,836.578377,62,736.642125,47,349.591633,173,345.953494,293,335.095565,336
6,429419,15237,940,28,94002,11266,08-12-2019,226000.0,133700,1.0,...,286.150889,23,236.638991,21,335.390755,109,617.593485,125,160.819866,182
7,430193,15122,870,45,87002,12445,09-08-2020,80000.0,68500,2.0,...,1346.471821,15,531.013014,57,529.601447,84,342.584995,153,36.344021,371
8,430698,15143,884,27,88405,100188,04-04-2018,295000.0,250800,1.0,...,1869.882119,1,1839.573113,9,2127.553934,6,783.373282,50,5551.757438,0
9,431954,15136,919,24,91901,11587,01-04-2017,280000.0,244700,2.0,...,1002.117208,6,895.534897,8,850.532377,39,268.493993,60,373.789015,165


In [9]:
results_df = pd.DataFrame()

## Only distance to nearest

Filter out all ``_count`` features.

In [10]:
structured_gis_dist = structured_gis[[col for col in structured_gis.columns if "_count" not in col]]
structured_gis_dist.shape

(9556, 100)

In [11]:
X_columns, data_sets, error_df = make_train_test(structured_gis_dist)
X, y, X_train, X_test, y_train, y_test, X_train_train, X_train_val, y_train_train, y_train_val = data_sets

(7167, 89): (5375, 89) + (1792, 89)
(7167,): (5375,) + (1792,)
(2389, 89)
(2389,)


### Linear regression

In [12]:
# model_01 = linear_model.LinearRegression()
model_01 = linear_model.Lasso()
# model_01 = linear_model.Ridge()
model_01.fit(X_train, y_train)

Lasso()

In [13]:
y_pred_01 = model_01.predict(X_test)
metrics = get_metrics(y_test, y_pred_01)

MAE:  39367
RMSE: 55089
MAPE: 24.25%
R^2:  0.829


Cross validation

In [14]:
results_df["Linear: S+D"] = cross_validation(model_01, X, y)

  0%|          | 0/5 [00:00<?, ?it/s]

MAE:  41501
RMSE: 58637
MAPE: 26.38%
R^2:  0.815


### Catboost

In [15]:
model_02 = CatBoostRegressor()
model_02.fit(X=X_train, y=y_train, verbose=False)

In [16]:
y_pred_02 = model_02.predict(X_test)
metrics = get_metrics(y_test, y_pred_02)

MAE:  28725
RMSE: 41493
MAPE: 16.59%
R^2:  0.903


Cross validation

In [17]:
results_df["Catboost: S+D"] = cross_validation(model_02, X, y)

  0%|          | 0/5 [00:00<?, ?it/s]

MAE:  29852
RMSE: 44221
MAPE: 17.72%
R^2:  0.895


## Distance to nearest and count in radius

Make train/test set for ``_dist`` and ``_count`` features

In [18]:
X_columns, data_sets, error_df = make_train_test(structured_gis)
X, y, X_train, X_test, y_train, y_test, X_train_train, X_train_val, y_train_train, y_train_val = data_sets

(7167, 125): (5375, 125) + (1792, 125)
(7167,): (5375,) + (1792,)
(2389, 125)
(2389,)


### Linear regression

In [19]:
# model_03 = linear_model.LinearRegression()
# model_03 = linear_model.Lasso()
model_03 = linear_model.Ridge()
model_03.fit(X_train, y_train)

Ridge()

In [20]:
y_pred_03 = model_03.predict(X_test)
metrics = get_metrics(y_test, y_pred_03)

MAE:  35561
RMSE: 49586
MAPE: 22.33%
R^2:  0.862


Cross validation

In [21]:
results_df["Linear: S+D+C"] = cross_validation(model_03, X, y)

  0%|          | 0/5 [00:00<?, ?it/s]

MAE:  37300
RMSE: 52906
MAPE: 23.78%
R^2:  0.85


### Catboost

In [22]:
model_04 = CatBoostRegressor()
model_04.fit(X=X_train, y=y_train, verbose=False)

In [23]:
y_pred_04 = model_04.predict(X_test)
metrics = get_metrics(y_test, y_pred_04)

MAE:  28098
RMSE: 40853
MAPE: 15.98%
R^2:  0.906


In [24]:
error_df["catboost"] = [test - pred for test, pred in zip(y_test, y_pred_04)]

Cross validation

In [25]:
results_df["Catboost: S+D+C"] = cross_validation(model_04, X, y)

  0%|          | 0/5 [00:00<?, ?it/s]

MAE:  29161
RMSE: 43369
MAPE: 17.15%
R^2:  0.899


### Only GIS features

Remove structured data

In [26]:
# remove house attributes
X_train_gis = X_train[:, 53:]
X_test_gis = X_test[:, 53:]

In [27]:
X_train.shape

(7167, 125)

### Linear regression

In [28]:
# model_05 = linear_model.LinearRegression()
# model_05 = linear_model.Lasso()
model_05 = linear_model.Ridge()
model_05.fit(X_train_gis, y_train)

Ridge()

In [29]:
y_pred_05 = model_05.predict(X_test_gis)
metrics = get_metrics(y_test, y_pred_05)

MAE:  70051
RMSE: 102158
MAPE: 40.91%
R^2:  0.413


Cross validation

In [30]:
results_df["Linear: D+C"] = cross_validation(model_05, X[:, 52:], y)

  0%|          | 0/5 [00:00<?, ?it/s]

MAE:  69060
RMSE: 101598
MAPE: 41.3%
R^2:  0.445


### Catboost

In [31]:
model_06 = CatBoostRegressor()
model_06.fit(X=X_train_gis, y=y_train, verbose=False)

In [32]:
y_pred_06 = model_06.predict(X_test_gis)
metrics = get_metrics(y_test, y_pred_06)

MAE:  51190
RMSE: 77443
MAPE: 29.83%
R^2:  0.662


Cross validation

In [33]:
results_df["Catboost: D+C"] = cross_validation(model_06, X[:, 52:], y)

  0%|          | 0/5 [00:00<?, ?it/s]

MAE:  50590
RMSE: 77057
MAPE: 29.71%
R^2:  0.681


## Results

In [34]:
results_df.index = ["MAE", "RMSE", "MAPE", "R^2"]
# reorder columns
results_df = results_df[["Linear: D+C", "Linear: S+D", "Linear: S+D+C", "Catboost: D+C", "Catboost: S+D", "Catboost: S+D+C"]]
results_df.to_csv(
    PATH + f"results/structured_gis_{COUNT_RADIUS}_results.csv", index=False)
print(f"Results for a count radius of {COUNT_RADIUS}m.")
results_df.head()

Results for a count radius of 3500m.


,Linear: D+C,Linear: S+D,Linear: S+D+C,Catboost: D+C,Catboost: S+D,Catboost: S+D+C
MAE,69060.000,41501.000,37300.00,50590.000,29852.000,29161.000
RMSE,101598.000,58637.000,52906.00,77057.000,44221.000,43369.000
MAPE,41.300,26.380,23.78,29.710,17.720,17.150
R^2,0.445,0.815,0.85,0.681,0.895,0.899


## Spatial out-of-sample test

Calculate median latitude and longitude

In [35]:
soos_df = structured_gis.copy()

coords_median = soos_df.loc[:, "latitude":"longitude"].median()
lat_median = coords_median.loc["latitude"]
long_median = coords_median.loc["longitude"]
coords_median

latitude     40.441981
longitude   -79.987716
dtype: float64

Split data into 4 quadrants along median latitude and longitude.

In [36]:
quadrants = []

quadrant_1 = soos_df[(soos_df["latitude"] >= lat_median) & (
    soos_df["longitude"] >= long_median)]
quadrants.append(quadrant_1)
print(quadrant_1.shape)

quadrant_2 = soos_df[(soos_df["latitude"] >= lat_median) & (
    soos_df["longitude"] < long_median)]
quadrants.append(quadrant_2)
print(quadrant_2.shape)

quadrant_3 = soos_df[(soos_df["latitude"] < lat_median) & (
    soos_df["longitude"] < long_median)]
quadrants.append(quadrant_3)
print(quadrant_3.shape)

quadrant_4 = soos_df[(soos_df["latitude"] < lat_median) & (
    soos_df["longitude"] >= long_median)]
quadrants.append(quadrant_4)
print(quadrant_4.shape, end="\n\n")

row_sum = quadrant_1.shape[0] + quadrant_2.shape[0] + quadrant_3.shape[0] + quadrant_4.shape[0]
print(f"{row_sum, quadrant_1.shape[1]}")

(2487, 136)
(2291, 136)
(2487, 136)
(2291, 136)

(9556, 136)


Create df for storing prediction errors

In [37]:
quadrants_df = pd.concat(quadrants, ignore_index=True)

error_df_soos = pd.DataFrame(
    data={"id": quadrants_df["_id"],
          "lat": quadrants_df["latitude"],
          "long": quadrants_df["longitude"],
          "prediction": 0,
          "error": 0})
error_df_soos.head(10)

,id,lat,long,prediction,error
0,431231,40.617547,-79.729775,0,0
1,432911,40.551018,-79.980841,0,0
2,235532,40.471628,-79.958138,0,0
3,434181,40.607946,-79.924800,0,0
4,436550,40.491269,-79.900028,0,0
5,437000,40.585733,-79.947796,0,0
6,239528,40.482701,-79.790070,0,0
7,440528,40.442026,-79.819429,0,0
8,484388,40.483444,-79.968761,0,0
9,441277,40.542078,-79.895429,0,0


Train model on 3 quadrants and predict the last one, so that every quadrant will be predicted once.

In [38]:
y_preds = []
errors = []
maes, rmses, mapes, r_squareds = [], [], [], []

for i, quadrant in enumerate(quadrants):
    train = pd.concat(quadrants[:i] + quadrants[i+1:])
    test = quadrants[i]
    
    train = train.drop(["_id", "PROPERTYZIP", "MUNICODE", "SCHOOLCODE", "NEIGHCODE", "SALEDATE",
                        "FAIRMARKETTOTAL", "latitude", "longitude", "SALEYEAR"], axis=1)
    test = test.drop(["_id", "PROPERTYZIP", "MUNICODE", "SCHOOLCODE", "NEIGHCODE", "SALEDATE",
                      "FAIRMARKETTOTAL", "latitude", "longitude", "SALEYEAR"], axis=1)
    
    X_train = train.drop(["SALEPRICE"], axis=1).to_numpy()
    y_train = train["SALEPRICE"].to_numpy()
    
    X_test = test.drop(["SALEPRICE"], axis=1).to_numpy()
    y_test = test["SALEPRICE"].to_numpy()
    
    model_cv = CatBoostRegressor()
    model_cv.fit(X=X_train, y=y_train, verbose=False)
    
    y_pred_cv = model_cv.predict(X_test)
    y_preds.extend(y_pred_cv)
    errors.extend([test - pred for test, pred in zip(y_test, y_pred_cv)])
    
    print(f"Quadrant: {i+1}")
    mae, rmse, mape, r_squared = get_metrics(y_test, y_pred_cv)
    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)
    r_squareds.append(r_squared)
    
    print("")

error_df_soos["prediction"] = y_preds
error_df_soos["error"] = errors
    
print("Average:")
print(f"MAE:  {round(np.mean(maes))}")
print(f"RMSE: {round(np.mean(rmses))}")
print(f"MAPE: {round(np.mean(mapes), 2)}%")
print(f"R^2:  {round(np.mean(r_squareds), 3)}")

Quadrant: 1
MAE:  50589
RMSE: 76877
MAPE: 29.53%
R^2:  0.758

Quadrant: 2
MAE:  45977
RMSE: 70358
MAPE: 19.77%
R^2:  0.775

Quadrant: 3
MAE:  34776
RMSE: 48192
MAPE: 17.05%
R^2:  0.821

Quadrant: 4
MAE:  35392
RMSE: 48389
MAPE: 33.12%
R^2:  0.741

Average:
MAE:  41684
RMSE: 60954
MAPE: 24.87%
R^2:  0.773


In [39]:
error_df_soos.head(10)

,id,lat,long,prediction,error
0,431231,40.617547,-79.729775,78573.362064,40426.637936
1,432911,40.551018,-79.980841,228202.748260,-45702.748260
2,235532,40.471628,-79.958138,469147.274583,152852.725417
3,434181,40.607946,-79.924800,225149.648685,49850.351315
4,436550,40.491269,-79.900028,239808.792878,200191.207122
5,437000,40.585733,-79.947796,183098.019137,56801.980863
6,239528,40.482701,-79.790070,266564.108333,-108564.108333
7,440528,40.442026,-79.819429,182206.223803,-28458.223803
8,484388,40.483444,-79.968761,164095.665761,-20595.665761
9,441277,40.542078,-79.895429,330457.795987,126292.204013


In [40]:
error_df_soos.to_csv(PATH+"results/errors_soos_gis.csv", index=False)

## Exploring solution

In [41]:
print(f"Intercept: {model_03.intercept_}")
feature_coef_df = pd.DataFrame(data={"feature": X_columns[53:], "coef": model_03.coef_[53:]})
feature_coef_df["coef"] = feature_coef_df["coef"].apply(lambda x: round(x, 2))

Intercept: -267481.0350934826


Most and least valuable POI based on count in vicinity

In [42]:
feature_coef_count = feature_coef_df[feature_coef_df["feature"].str.contains("count")]

neg_sorted = feature_coef_df.sort_values(by=["coef"], ascending=False).head(10)
pos_sorted = feature_coef_df.sort_values(by=["coef"]).head(10)

df1_styler = pos_sorted.style.set_table_attributes("style='display:inline'").set_caption('Most valuable POI')
df2_styler = neg_sorted.style.set_table_attributes("style='display:inline'").set_caption('Least valuable POI')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,feature,coef
43,libraries_count,-5294.860000
37,post_offices_count,-4803.510000
7,wic_vendors_count,-3792.970000
27,public_buildings_count,-1951.520000
57,polling_places_count,-1797.890000
47,supermarkets_count,-1668.480000
55,child_care_centers_count,-1369.260000
61,schools_count,-1200.110000
67,restaurants_count,-532.680000
15,park_and_rides_count,-517.790000


Most and least valuable POI based on distance to home

In [43]:
feature_coef_dist = feature_coef_df[feature_coef_df["feature"].str.contains("dist")]

neg_sorted = feature_coef_dist.sort_values(by=["coef"], ascending=False).head(10)
pos_sorted = feature_coef_dist.sort_values(by=["coef"]).head(10)

df1_styler = pos_sorted.style.set_table_attributes("style='display:inline'").set_caption('Most valuable POI')
df2_styler = neg_sorted.style.set_table_attributes("style='display:inline'").set_caption('Least valuable POI')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,feature,coef
38,banks_dist,-7.250000
16,bars_dist,-4.460000
44,doctors_offices_dist,-4.460000
60,schools_dist,-3.410000
36,post_offices_dist,-3.380000
62,apartment_buildings_dist,-3.120000
42,libraries_dist,-2.720000
46,supermarkets_dist,-2.480000
18,bike_share_stations_dist,-2.030000
8,barbers_dist,-1.960000


Explore feature importance of best model

In [44]:
feature_importance_df = pd.DataFrame(data={"feature": X_columns[53:],
                                           "importance": model_04.get_feature_importance()[53:]})
feature_importance_df.sort_values(by=["importance"], ascending=False).head(10)

,feature,importance
25,nursing_homes_count,2.204712
2,senior_centers_dist,2.094696
39,banks_count,1.663273
4,health_centers_dist,1.420168
21,affordable_housing_count,1.242234
18,bike_share_stations_dist,1.110660
0,universities_dist,1.063690
17,bars_count,1.037626
69,community_nonprofit_orgs_count,1.031040
20,affordable_housing_dist,0.971309
